In [ ]:
import os
import re
from datetime import datetime
from os import path
from typing import List, Dict

from commonroad.common.util import Interval

from BatchConversion.BatchConverter import BatchConverter, BatchConversionResult
from OpenSCENARIO2CR.ConversionAnalyzer.EAnalyzer import EAnalyzer
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.EsminiWrapperProvider import EsminiWrapperProvider
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter, EFailureReason
from OpenSCENARIO2CR.util.AbsRel import AbsRel
from OpenSCENARIO2CR.util.PlanningProblemBuilder import PlanningProblemSetBuilder

In [ ]:
esmini_wrapper = EsminiWrapperProvider(preferred_version="v2.26.5").provide_esmini_wrapper()
esmini_wrapper.max_time = 120.0
esmini_wrapper.grace_time = 1.0
esmini_wrapper.ignored_level = EStoryBoardElementType.ACT
esmini_wrapper.log_to_console = False
esmini_wrapper.log_to_file = False
esmini_wrapper.random_seed = 0

In [ ]:
pps_builder = PlanningProblemSetBuilder()
pps_builder.goal_state_time_step = AbsRel(Interval(-10, 0), False)
pps_builder.goal_state_position_length = 50
pps_builder.goal_state_position_width = 10
pps_builder.goal_state_position_use_ego_rotation = True
pps_builder.goal_state_orientation = None
# pps_builder.goal_state_orientation = AbsRel(AngleInterval(-0.8, 0.8), False)
pps_builder.goal_state_velocity = None
# pps_builder.goal_state_velocity = AbsRel(Interval(-10, 10), False)

In [ ]:
converter = Osc2CrConverter()

converter.sim_wrapper = esmini_wrapper
converter.pps_builder = pps_builder

converter.dt_cr = 0.1
converter.keep_ego_vehicle = True
converter.trim_scenario = False
converter.use_implicit_odr_file = True
converter.analyzers = [EAnalyzer.STL_MONITOR, EAnalyzer.DRIVABILITY_CHECKER]

converter.dt_sim = 0.01
converter.odr_file_override = None
converter.ego_filter = re.compile(r".*ego.*", re.IGNORECASE)


In [ ]:
# Configured see the previous cells
batch_converter = BatchConverter(converter)
# If the default parameters are sufficient the following line is sufficient
# batch_converter = BatchConverter(Osc2CrConverter())

In [ ]:
from os.path import expanduser

batch_converter.discover_files(expanduser("~"), re.compile(r".*\.xosc", re.IGNORECASE), recursively=True)

In [ ]:
time_conversion_started = datetime.now()
results: Dict[str, BatchConversionResult] = batch_converter.run_batch_conversion(num_worker=0)
time_conversion_finished = datetime.now()

In [ ]:
from commonroad.scenario.scenario import Tag
from commonroad.common.file_writer import CommonRoadFileWriter, OverwriteExistingFile
import yaml

storage_path = "results/{}".format(time_conversion_finished.isoformat(sep="_", timespec="seconds"))
store = True

if store:
    if not path.exists(storage_path):
        os.mkdir(storage_path)
    statistics_dict = {}
    written_scenarios: List[str] = []
    for scenario_path, result in results.items():
        basename = path.basename(scenario_path)
        path_no_extension = path.join(storage_path, path.splitext(basename)[0])
        if path_no_extension in written_scenarios:
            i = 1
            while True:
                if (new_path := path_no_extension + f"-{i}") not in written_scenarios:
                    path_no_extension = new_path
                    break
                else:
                    i += 1
        written_scenarios.append(path_no_extension)

        if result.without_exception:
            if isinstance(result.conversion_result, EFailureReason):
                statistics_dict[path_no_extension] = ("failure", result.conversion_result.name)
            else:
                scenario, planning_problem_set, statistics = result.conversion_result
                statistics_dict[path_no_extension] = ("success", statistics.to_dict())
                CommonRoadFileWriter(
                    scenario=scenario,
                    planning_problem_set=planning_problem_set,
                    author="ADD AUTHOR HERE",
                    affiliation="ADD AFFILIATION HERE",
                    source="ADD SOURCE HERE",
                    tags={Tag.SIMULATED},
                ).write_to_file(path_no_extension + ".xml", OverwriteExistingFile.ALWAYS)
        else:
            statistics_dict[path_no_extension] = ("error", result.exception.to_dict())

    with open(path.join(storage_path, "statistics.yml"), "w") as file:
        yaml.safe_dump(statistics_dict, file)


In [ ]:
from OpenSCENARIO2CR.ConversionAnalyzer.ErrorAnalysisResult import ErrorAnalysisResult

errors: Dict[ErrorAnalysisResult, int] = {}
for result in results.values():
    if not result.without_exception:
        if result.exception in errors:
            errors[result.exception] += 1
        else:
            errors[result.exception] = 1

for error, count in errors.items():
    print(f"{count}\n{error.traceback_text}")
    print("\n" * 3)

In [ ]:
from OpenSCENARIO2CR.ConversionAnalyzer.ErrorAnalysisResult import ErrorAnalysisResult

errors: Dict[ErrorAnalysisResult, int] = {}
for result in results.values():
    if result.without_exception:
        if not isinstance(result.conversion_result, EFailureReason):
            scenario, planning_problem_set, statistics = result.conversion_result
            for analysis_type, analysis_result in statistics.analysis.items():
                for vehicle_name, vehicle_result in analysis_result.items():
                    if isinstance(vehicle_result, ErrorAnalysisResult):
                        if vehicle_result in errors:
                            errors[vehicle_result] += 1
                        else:
                            errors[vehicle_result] = 1

for error, count in errors.items():
    print(f"{count}\n{error.traceback_text}")
    print("\n" * 3)